# Multiple Regression

`minutes =  α + β1friends + β2work_hours + β3phd + ε`

A `dummy variable` is a way to turn a category (yes/no) into a number so the regression model can use it.


We introduced a 'dummy variable' that equals to 1 for users with phDs and 0 for the users without, after which it's just as numeric as the other variables


How it works:

If phd = 1 → β3 is added to the prediction.

If phd = 0 → β3 is NOT added (because anything × 0 = 0).

- Simple meaning: <br>
β3 tells us how much extra (or less) minutes are expected for someone with a PhD compared to someone without one.

## The Model

For multiple regression model assumes that:

`yi = α + β1x_i_1+...+βkx_i_k + εi`

In multiple regression the vectors of parameteres are usually called (β).

beta = [alpha, beta_1,....., beta_k]

x_i = [1, x_i1,...., x_ik]

- We add a 1 at the start of every data vector so the constant term can be included.

In [3]:
from typing import List

Vector = List[float]

def dot(v:Vector,w:Vector)->Vector:
    assert len(v) == len(w), "different sizes"
    return sum(v_i * w_i for v_i,w_i in zip(v,w))

def predict(x:Vector, beta:Vector)->float:
    """ assumes that the first element of x is 1 """
    return dot(x,beta)

## Fitting the Model

In case of Multiple Regression

`y = α + β₁*x₁ + β₂*x₂ + … + βk*xk + ε`

we have (n) number of beta, so calculating them and finding exact solution by hand is too difficult.

We will need to use gradient descent to find the values of beta and minimize the sum of the squared errors.


In [4]:
from typing import List

def error(x:Vector, y:float, beta:Vector)-> float:
    return predict(x,beta) - y

def squared_error(x:Vector, y:float, beta:Vector)-> float:
    return error(x,y,beta)**2

x = [1,2,3]
y = 30
beta = [4,4,4]  # so prediction  = 4+8+12 = 24

assert error(x,y,beta) == -6
assert squared_error(x,y,beta) == 36

Using calculus to compute the gradient 

In [6]:
def sqerror_gradient(x:Vector,y:float,beta:Vector)->Vector:
    err = error(x,y,beta)
    return [2*err*x_i for x_i in x]

assert sqerror_gradient(x,y,beta) == [-12, -24, -36]

#### Below code is from Ch-4 and Ch-8

In [ ]:
def vector_sum(vectors: List[Vector]) -> Vector:
    # check that vectors is not empty
    assert vectors, "no vectors provided!"

    # check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes"
    return [sum(vector[i] for vector in vectors) for i in range(num_elements)]

def vector_mean(vectors: List[Vector])-> Vector:
    num_elements = len(vectors)
    return [(1/num_elements)*i for i in vector_sum(vectors)]

def add(v:Vector, w:Vector)-> Vector:
    return[v_i+w_i for v_i,w_i in zip(v,w)]

# Vector Multiplication
def scalar_multiplication(c:float,v:Vector)-> Vector:
    return[c*v_i for v_i in v]

def gradient_step(v:Vector, gradient:Vector, step_size: float) -> Vector:
    """ Moves 'step size' in the `gradient` direction from `v` """
    assert len(v) == len(gradient)
    step = scalar_multiplication(step_size, gradient)
    return add(v,step)


Now, write the least_squares_fit function that can work with any dataset

In [ ]:
import random
import tqdm

def least_squares_fit(xs:List[Vector],
                      ys:List[float],
                      learning_rate: float = 0.001,
                      num_steps:int = 1000,
                      batch_size:int = 1) -> Vector:
    """
    Find the beta the minimizes the sum of squared errors
    assuming the model y = dot(x, beta)
    """

    # start with a random guess
    guess = [random.random() for _ in xs[0]]

    for _ in tqdm.trange(num_steps, desc="least_squares_fit"):
        for start in range (0,len(xs),batch_size):
            batch_xs = xs[start:start+batch_size]
            batch_ys = ys[start:start+batch_size]

            gradient = vector_mean([sqerror_gradient(x,y,guess)]
                                   for x,y in zip(batch_xs,batch_ys))
            guess = gradient_step(guess,gradient,-learning_rate)
    
    return guess

## Goodness of Fit

In [ ]:
def mean(xs: List[float])-> float:
    return sum(xs)/len(xs)

def de_mean(xs:List[float]) -> List[float]:
    x_bar = mean(xs)
    return[x-x_bar for x in xs]

def total_sum_of_squares(y:Vector) -> float:
    """ The total sqaured variation of y_i's from their mean """
    return sum(v**2 for v in de_mean(y))

# R-squared

def multiple_r_squared(xs:List[Vector],ys:Vector, beta:Vector)->float:
    sum_of_squared_errors = sum(error(x,y,beta)**2
                                for x,y in zip(xs,ys))
    
    return 1.0 - sum_of_squared_errors/total_sum_of_squares(ys)

Keep in mind that adding new variables to a regression will necessarily increase the R-squared. 

After all, the simple regression model is just the special case of multiple regression model where the coefficients on "work hours" and "PhD" both equal 0.

## Digression: The Bootstrap

In [ ]:
# Imagine we have sample of n data points, generated by some(unknown to us) distribution

data = get_sample(num_points=n)    # Here, get_sample() is some function 

from typing import TypeVar, Callable

X = TypeVar('X')            # Generic type for data
Stat = TypeVar('Stat')      # Generic type for "statistics"
def bootstrap_sample(data:List[X])-> List[X]:
    """ randomly samples len(data) elements with replacement """
    return [random.random(data) for _ in data]

def bootstrap_statistics(data: List[X],
                         stats_fn: Callable[[List[X]],Stat],
                         num_samples: int
                         )-> List[Stat]:
    
    """ evaluates stats_fn on num_samples bootstrap samples from data """
    return [stats_fn(bootstrap_sample(data)) for _ in range(num_samples)]


# For examples lets consider the following datasets:

# 101 Points all very close to 100
close_to_100 = [99.5 + random.random() for _ in range(101)]

# 101 points, 50 of them near 0, 50 of them near 200

far_from_100 = ([99.5 + random.random()]+
                [random.random() for _ in range(50)] + [200 + random.random() for _ in range(50)])

""" You compute the medians of the two datasets, both will be very close to 100 """
import math
# VARIANCE

def de_mean(xs:List[float]) -> List[float]:
    x_bar = mean(xs)
    return[x-x_bar for x in xs]

def sum_of_squares(xs:List[float])-> float:
    return sum(x_i*x_i for x_i in xs)

def variance(xs:List[float])->float:
    assert len(xs) >= 2 , "variance requires at least two elements"

    n = len(xs)
    deviations = de_mean(xs)
    return(sum_of_squares(deviations)/(n-1))

def standard_deviation(xs:List[float])-> float:
    ''' The standard deviation is the square root of the variance '''
    return math.sqrt(variance(xs))
def _median_odd(xs:List[float])->float:
    return sorted(xs)[len(xs)//2]                 # '//' -> it is floor division 
    
def _median_even(xs:List[float]) -> float:
    sorted_xs = sorted(xs)
    h_midpoint  = len(xs)//2
    return ((sorted_xs[h_midpoint-1]+sorted_xs[h_midpoint])/2)

def median(c:List[float]) -> float:
    return (_median_even(c) if len(c)%2==0 else _median_odd(c))

# Now, execute:
medians_close = bootstrap_statistics(close_to_100,median,100)
medians_far  = bootstrap_statistics(far_from_100,median,100)

assert standard_deviation(medians_close) < 1
assert standard_deviation(medians_far) > 90

## Standard Errors of Regression Coefficients

In [ ]:
from typing import Tuple
import datetime

def estimate_sample_beta(pairs:List[Tuple[Vector,float]]):
    x_sample = [x for x,_ in pairs]
    y_sample = [y for y,_ in pairs]

    learning_rate  = 0.0001

    beta = least_squares_fit(x_sample,y_sample,learning_rate,5000,25)
    print("bootstrap sample",beta)
    return beta

random.seed(0)

# This will take a couple of minutes!
bootstrap_betas =  bootstrap_statistics(list(zip(inputs,daily_minutes_good)),estimate_sample_beta,100)

# After which we can estimate the standard deviation of each coefficient:

bootstrap_standard_errors = [
    standard_deviation([beta[i] for beta in bootstrap_betas]
                       for i in range(4))
]

print(bootstrap_standard_errors)

We can use to test hypotheses such as "does βi does 0?" Under the null hypothesis βi=0.

Here we use, t-statistics: 

`tj =ˆβj / ˆσj` 

Here, `ˆβj = estimated of βj`  <br> <br>
    `ˆσj  = estimate of its standard error`

- t = estimated effect / uncertainty in that estimate

So:

- Large t → strong evidence the variable matters

- Small t → could just be random noise


In t-distribution, we compute the t-value, it follows something called the `Student's t-distribution` <br>
which depend on : <br> <br>
           ` degress of freedom = n - k`

Where:

- n = number of data points

- k = number of variables

More data → higher degrees of freedom → better reliability.   

A p-value tells us:

“If the true coefficient were actually 0, how likely is it that we’d see a result this extreme?”

-  Small p-value(usually (<0.005)) → Reject H₀ (β = 0)
-  Large p-value → Fail to reject H₀
-  

When the dataset is large, the t-distribution becomes almost identical to the normal distribution.

So instead of using a complicated t function, we can safely use the normal CDF (normal probability function).

-  Easier
-  Faster
-  Still accurate when n is large

In [ ]:
def normal_cdf(x:float, mu:float = 0, sigma:float = 1)-> float:
    return(1+math.erf((x-mu)/math.sqrt(2)/sigma))/2

def p_value(beta_hat_j:float, sigma_hat_j:float)->float:
    if beta_hat_j>0:
        # If coefficient is positive, we need to compute the twice the probability of seeing an even *larger* value
        return 2 * (1-normal_cdf(beta_hat_j/sigma_hat_j))
    else:
        #otherwise twice the probability of seeing a *smaller* value
        return 2 * normal_cdf(beta_hat_j/sigma_hat_j)
    

assert p_value(30.58, 1.27)   < 0.001  # constant term
assert p_value(0.972, 0.103)  < 0.001  # num_friends
assert p_value(-1.865, 0.155) < 0.001  # work_hours
assert p_value(0.923, 1.249)  > 0.4    # phd

Testing bigger hypotheses (F-test)

Sometimes we don’t want to test just one variable.

We want bigger questions like:

-  “Does at least one variable matter?”
  
-  “Are these variables equal?
  

Simple idea of an F-test:
-  t-test → checks ONE coefficient

-  F-test → checks MULTIPLE coefficients together

## Regularization 

It is an approach in which we add to the error term a penalty that gets larger as beta gets larger. 
We then minimize the combined error and penalty.

For example, in ridge regression, we add a penalty proportional to the sum of squared of beta_i(except the typically we don't penalize beta_0 the constant term)

In ridge regression, the loss is:

`Loss=Squared Error+Ridge Penalty`

Ridge Penalty is :

`Penalty=α(β1^2 ​+ β2^2 ​+⋯+βn^2​)`

`Squared Error= (y−y^​)2 = (y−(β0​+β1​x1​+β2​x2​+⋯+βn​xn​))^2`

so:

`Total loss = (y−(β0​+β1​x1​+β2​x2​+⋯+βn​xn​))^2 + α(β1^2 ​+ β2^2​+⋯+βn^2​)`

Ridge gradient of β0 = 0

and Gradient of Ridge penalty = **2αbj**


In [ ]:
# alpha is a *hyperparameter* controlling how harsh the penalty is

def ridge_penalty(beta:Vector, alpha:float)->float:
    return alpha * dot(beta[1:],beta[1:])

def squared_error_ridge(x:Vector,
                        y:float,
                        beta: Vector,
                        alpha:float)->float:
    """ estimate error plus ridge penalty on beta """
    return error(x, y, beta)**2 + ridge_penalty(beta,alpha)

Now use gradient descent

In [ ]:
def add(v:Vector, w:Vector)-> Vector:
    assert len(v)==len(w), "Vectors must be in same length"
    return[v_i+w_i for v_i,w_i in zip(v,w)]

def ridge_penalty_gradient(x:Vector,
                           y:float,
                           beta:Vector,
                           alpha:float)-> Vector:
    """ Gradient of just the ridge penalty """
    return [0.] + [2 * alpha * beta_j for beta_j in beta[1:]]

def sqerror_ridge_gradient(x:Vector, y:float, beta:Vector, alpha:float)->Vector:
    return add(sqerror_gradient(x,y,beta),
               ridge_penalty_gradient(beta,alpha))

#### Lasso Regression

`Lasso Penalty= α(∣β1∣ + ∣β2∣+⋯+∣βn∣)`

Uses absolute value instead of square.

Whereas ridge penalty shrank the coefficients overall but Lasso can force some coefficients to be exactly 0, effectively dropping unimportant features.

Lasso uses the absolute value, so the gradient is not smooth at 0:

$$
\frac{\partial}{\partial \beta_j} |\beta_j| =
\begin{cases} 
1 & \text{if } \beta_j > 0 \\[2mm]
-1 & \text{if } \beta_j < 0 \\[1mm]
\text{undefined} & \text{if } \beta_j = 0
\end{cases}
$$

This **non-differentiability at 0** makes it hard to solve using standard gradient descent from scratch.


In [ ]:
def lasso_penalty(beta, alpha):
    return alpha * sum(abs(beta_i for beta_i in beta[1:]))